# Figures: *Analysis of the five-dimensional phase space distribution of an ion beam using direct, efficient, high-resolution measurements*

In [ ]:
import sys
import os
import itertools
import importlib

import h5py
import numpy as np
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from plotly import graph_objects as go
from matplotlib import pyplot as plt
from ipywidgets import interactive
from ipywidgets import widgets
import proplot as pplt

sys.path.append('/home/46h/psdist/')
from psdist import dist as bd
from psdist import image as bi
from psdist import plotting as mplt
from psdist import utils

sys.path.append('/home/46h/btf-data-analysis/')
from tools import analysis as ba

## Settings 

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['font.family'] = 'serif'
pplt.rc['font.serif'] = 'Times New Roman'

def save(figname, **kws):
    ext = 'pdf'
    figpath = './figures'
    plt.savefig(os.path.join(figpath, figname + '.' + ext), **kws)

## Load data 

In [ ]:
# folder = '/home/46h/btf-data-analysis/scan3d/scan-xxpy-image-ypdE/_saved/2021-12-03-VS06'
# folder = '/home/46h/btf-data-analysis/scan3d/scan-xxpy-image-ypdE/_saved/2022-07-15-VS06'
folder = '/home/46h/btf-data-analysis/scan3d/scan-xxpy-image-ypdE/_saved/2022-06-26-VS06'

Load the phase space density array `f` and grid coordinates `coords`.

In [ ]:
info = utils.load_pickle(os.path.join(folder, 'info.pkl'))
filename = info['filename']
coords = utils.load_stacked_arrays(os.path.join(folder, f'coords_{filename}.npz'))
shape = tuple([len(c) for c in coords])
f = np.memmap(os.path.join(folder, f'f_{filename}.mmp'), shape=shape, dtype='float', mode='r')

print('f.shape:', shape)

Give units to the coordinates. Default is mm, mrad, MeV.

In [ ]:
coords[4] *= 1000.0

dims = ["x", "x'", "y", "y'", "w"]
units = ["mm", "mrad", "mm", "mrad", "keV"]
dims_units = [f'{d} [{u}]' for d, u in zip(dims, units)]

Center the coordinates on the centroid calculated from the profiles.

In [ ]:
profs = []
for i in range(5):
    prof = bi.project(f, axis=i)
    prof = prof / np.sum(prof)
    coords[i] = coords[i] - np.average(coords[i], weights=prof)
    profs.append(prof)

Crop the array.

In [ ]:
crop = (
    (0, f.shape[0]),
    (10, f.shape[1] - 10),
    (0, f.shape[2]),
    (17, f.shape[3] - 14),
    (10, f.shape[4] - 15),
)

fig, axes = pplt.subplots(ncols=5, figwidth=7, spanx=False, figheight=None)
for i, ax in enumerate(axes):
    ax.plot(profs[i], color='black')
    ax.axvspan(crop[i][0], crop[i][1] - 1, color='black', alpha=0.1)
    ax.format(xlabel=dims[i] + ' [pixel]')
# axes.format(yscale='log')
plt.show()

In [ ]:
ind = tuple([slice(c[0], c[1]) for c in crop])
f = f[ind]
coords = [c[ind[i]] for i, c in enumerate(coords)]

Normalize `f` to the range [0, 1].

In [ ]:
f_min = np.min(f)
if f_min < 0.0:
    print(f'min(f) = {f_min}')
    print('Clipping to zero.')
    f = np.clip(f, 0.0, None)
f = f / np.max(f)

print(f'f_min = {np.min(f)}')
print(f'f_max = {np.max(f)}')

Apply a threshold to `f`.

In [ ]:
f[f < 10.0**-3.5] = 0

## Rectangular slices

### 1D projections 

`slice_type` can be 'int', in which case only one index is selected along the axis, or 'range', in which case a range of indices are selected.

In [ ]:
mplt.interactive_proj1d(
    f, coords=coords, dims=dims, units=units, default_ind=4,
    kind='line',  # {'line', 'step', 'bar'}
    slice_type='int',  # {'int', 'range'}
)

Zoom in on the center of transverse phase space.

In [ ]:
pplt.rc['font.family'] = 'sans-serif'
pplt.rc['font.sans-serif'] = 'DejaVu Sans'
pplt.rc['mathtext.fontset'] = 'dejavusans'

In [ ]:
ind_center = [np.argmin(np.abs(c)) for c in coords]

fig, axes = pplt.subplots(ncols=4, figwidth=4.5, figheight=1.1)
axes.format(xlabel="Energy [keV]", yticklabels=[], xlim=(-65.0, 65.0))
order = [0, 2, 1, 3]
for i, ax in enumerate(axes):
    idx = bi.make_slice(4, order[:i+1], [ind_center[k] for k in order[:i+1]])
    frac = np.sum(f[idx]) / np.sum(f)
    print(f'frac_volume = {(f[idx].size / f.size)}')
    pw = bi.project(f[idx], 3 - i)
    pw = pw / np.sum(pw)
    ax.bar(coords[4], pw, color='black', width=1.0)
    title = ""
    for j in order[:i+1]:
        title += r"${} \approx$".format(dims[j])
    title += "0"
    # title += f"\n({(100.0 * frac):.3f}%)"
    ax.format(title=title, title_kw=dict(fontsize='medium'))
axes.format(ylim=(0.0, axes[0].get_ylim()[1]), xspineloc='bottom', yspineloc='neither')
save('energy_transverse_slice')

### 2D projections 

In [ ]:
prof_kws = dict(lw=0.5, alpha=0.7, color='white', scale=0.12)  # for 1D profiles

mplt.interactive_proj2d(
    f, coords=coords, dims=dims, units=units, prof_kws=prof_kws, 
    slice_type='int',  # {'int', 'range'}
    cmap='mono',
)

### Crazy plot

In [ ]:
axis_proj = (0, 2, 3, 4)
_f = bi.project(f, axis=axis_proj)  
_f = _f / np.max(_f)

In [ ]:
# Now work in 4D space x-y-y'-w
axis_view = (0, 1)
axis_slice = (2, 3)
nrows = ncols = 9

ind_slice = []
for i, n in zip(axis_slice, [nrows, ncols]):
    pad = int(0.2 * _f.shape[i])
    ii = np.linspace(pad, _f.shape[i] - 1 - pad, nrows)
    ii = ii.astype(int)
    ii = list(ii)
    ind_slice.append(ii)

In [ ]:
idx = 4 * [slice(None)]
for axis, ind in zip(axis_slice, ind_slice):
    idx[axis] = ind
    _f = _f[tuple(idx)]
    idx[axis] = slice(None)
_f = _f / np.max(_f)

_fx = bi.project(f, axis=(0, 2, 3))[:, :, ind_slice[0]]
_fy = bi.project(f, axis=(0, 2, 4))[:, :, ind_slice[1]]
print(_f.shape, _fx.shape, _fy.shape)

In [ ]:
_coords = [coords[i] for i in axis_proj]
for axis, ind in zip(axis_slice, ind_slice):
    _coords[axis] = [_coords[axis][i] for i in ind]
_dims = [dims[i] for i in axis_proj]
_units = [units[i] for i in axis_proj]
_labels = [f'{dim} [{unit}]' for dim, unit in zip(_dims, _units)]
print(_dims)
print(_units)
print(_labels)
print([len(c) for c in _coords])

In [ ]:
space = 0.35
hspace = nrows * [space]
wspace = ncols * [space]
hspace[-1] = wspace[-1] = 5.0 * space
fig, axes = pplt.subplots(
    ncols=ncols+1, nrows=nrows+1, figwidth=8.5,
    hspace=hspace, wspace=wspace,
    share=False,
    xticks=[], yticks=[],
)
plot_kws = dict(
    # profx=True, profy=True, prof_kws=dict(lw=0.4, alpha=0.5, kind='step'),
    thresh=10.0**-3.5,
    thresh_type='frac',
)
for i in range(nrows):
    for j in range(ncols):
        ax = axes[i, j]
        idx = bi.make_slice(_f.ndim, axis_slice, (i, j))
        mplt.plot_image(
            _f[idx],
            x=_coords[axis_view[0]], 
            y=_coords[axis_view[1]],
            ax=ax,
            vmax=None,
            **plot_kws
        )
for i, ax in enumerate(axes[:-1, -1]):
    mplt.plot_image(
        _fy[:, :, i],
        x=_coords[axis_view[0]], 
        y=_coords[axis_view[1]],
        ax=ax,
        **plot_kws
    )
for i, ax in enumerate(axes[-1, :-1]):
    mplt.plot_image(
        _fx[:, :, i],
        x=_coords[axis_view[0]], 
        y=_coords[axis_view[1]],
        ax=ax,
        **plot_kws
    )
mplt.plot_image(
    bi.project(f, (0, 2)),
    x=_coords[axis_view[0]], 
    y=_coords[axis_view[1]],
    ax=axes[-1, -1],
    **plot_kws
)
axes[-2, 0].format(xlabel=_dims[axis_view[0]], ylabel=_dims[axis_view[1]])
save(f'matrix_{nrows}x{ncols}')
plt.show()

In [ ]:
_cmap = pplt.Colormap('mono', right=0.75)
_cmap

In [ ]:
axis = (0, 2, 4)
_f = utils.project(f, axis=axis)
_f = _f / np.max(_f)
_coords = [coords[i] for i in axis]
_dims = [dims[i] for i in axis]
_units = [units[i] for i in axis]
_dims_units = [dims_units[i] for i in axis]

ncols = 5
# start_stop = [
#     (14, len(_coords[0]) - 1 - 14),
#     (19, len(_coords[1]) - 1 - 19),
#     (20, len(_coords[2]) - 1 - 20),
# ]
start_stop = [
    (9, len(_coords[0]) - 1 - 9),
    (10, len(_coords[1]) - 1 - 10),
    (25, len(_coords[2]) - 1 - 25),
]

for cmap in ['viridis', _cmap]:
    for i, ((start, stop), c) in enumerate(zip(start_stop, _coords)):
        fig, axes = pplt.subplots(ncols=ncols, nrows=1, figwidth=6.0)
        jj = np.linspace(start, stop, ncols).astype(int)
        for ax, j in zip(axes, jj):
            idx = utils.make_slice(_f.ndim, i, int(j))
            axis_view = [k for k in range(_f.ndim) if k != i]
            mplt.plot_image(
                _f[idx], 
                x=_coords[axis_view[0]], 
                y=_coords[axis_view[1]], 
                ax=ax,
                vmin=0.0, vmax=1.0,
                linewidth=0, rasterized=True,
                cmap=cmap,
                # colorbar=ax==axes[-1], 
                colorbar_kw=dict(
                    width=0.1, 
                    # ticks=[0, 1],
                ),
                profx=True, profy=True, prof_kws=dict(lw=0.4, alpha=0.5, color='white', scale=0.09),
            )
            if ax.get_xlim()[1] < ax.get_xlim()[0]:
                ax.format(xlim=sorted(ax.get_xlim()))
            ax.annotate(r"${} \approx {:.0f}$ [{}]".format(_dims[i], _coords[i][j], _units[i]), 
                        color='white', 
                        fontsize='small',
                        xy=(0.02, 0.98), verticalalignment='top', xycoords='axes fraction')
        axes.format(
            xlabel=_dims_units[axis_view[0]], 
            ylabel=_dims_units[axis_view[1]],
        )
        plt.savefig(f'_output/{_dims[axis_view[0]]}{_dims[axis_view[1]]}_{_dims[i]}slice_{cmap}')
        plt.show()

In [ ]:
for i, (start, stop) in enumerate(start_stop):
    print(_dims[i], _coords[i][start], _coords[i][stop])

Slice through energy.

In [ ]:
# Find vmax for each 3D image stack (x1-x2-w).
nslice = 6
inds = [(0, 1), (2, 3), (0, 2), (0, 3), (2, 1), (1, 3)]
ii = np.linspace(19, len(_coords[2]) - 1 - 22, nslice).astype(int)
vmaxs = []
for (j, k) in tqdm(inds):
    _f = utils.project(f[..., ii], axis=(j, k, 4))
    vmaxs.append(np.max(_f))

In [ ]:
# cmap = _cmap
cmap = 'viridis'
prof = False
space = 0.0 if cmap == _cmap else 0.5
fig, axes = pplt.subplots(ncols=6, nrows=nslice, figwidth=8.0, share=False, space=space)
axes.format(xticks=[], yticks=[])
for ax, (j, k) in zip(axes, inds):
    ax.format(title=f'{dims[j]}-{dims[k]}')
for row, i in enumerate(ii):
    axes[row, 0].format(ylabel=r"$w \approx {:.0f}$ [keV]".format(coords[4][i]))
    for col, (ax, (j, k)) in enumerate(zip(axes[row, :], inds)):
        image = utils.project(f[..., i], (j, k))
        mplt.plot_image(
            image,
            x=coords[j],
            y=coords[k],
            ax=ax,
            vmax=vmaxs[col],
            cmap=cmap,
            linewidth=0, rasterized=True,
            profx=prof, profy=prof, prof_kws=dict(lw=0.4, alpha=0.5, color='white', scale=0.09),
        )
plt.savefig(f'_output/transverse_wslice_stacked_{cmap}_{prof}')
plt.show()

In [ ]:
pwd

## Contour slices

We can also use contour slices — volumes defined by density contours.

### 1D projections

Plot the 1D projection of the distribution along any axis. The `4D thresh` slider applies a threshold to the remaining four dimensions before computing the projection. For example, if we are looking at the energy projection, `4D thresh` applies to the transverse phase space x-x'-y-y'. The density in this space is normalized to the range [0, 1], so a threshold of 0 selects all particles, while a threshold of 1 selects only the brightest pixel in x-x'-y-y'. The fraction of the beam selected is printed on the figure (`frac`).

In [ ]:
def proj1d_mask(f, thresh=0.1, fpr=None, normalize=True, return_frac=False, axis=0):
    axis_proj = [i for i in range(f.ndim) if i != axis]
    if fpr is None:
        fpr = utils.project(f, axis_proj)
    fpr = fpr / np.max(fpr)
    idx = np.where(fpr > thresh)
    frac = np.sum(fpr[idx]) / np.sum(fpr)
    idx = utils.make_slice(f.ndim, axis_proj, idx)    
    p = np.sum(f[idx], axis=int(axis == 0))
    if normalize:
        p = p / np.sum(p)
    if return_frac:
        return p, frac
    return p

In [ ]:
def update(thresh, dim):
    axis = dims.index(dim)
    p, frac = proj1d_mask(f, thresh, normalize=True, return_frac=True, axis=axis)
    
    fig, ax = pplt.subplots(figsize=(4, 1.5))
    ax.format(xlabel=dims_units[axis], title=f'frac = {frac:.2f}')
    ax.bar(coords[axis], p, color='black')
    plt.show()
    
    
thresh = widgets.FloatSlider(description='4D thresh', min=0.0, max=0.99, value=0.5, step=0.001)
interactive(update, thresh=thresh, dim=reversed(dims))

### 2D projections 

We can do the same thing with 2D projections, applying a threshold in the remaining three dimensions.

In [ ]:
def proj2d_mask(f, thresh=0.1, fpr=None, normalize=True, return_frac=False, axis=(2, 3)):
    # Compute the 3D mask.
    axis_proj = [i for i in range(f.ndim) if i not in axis]
    if fpr is None:
        fpr = utils.project(f, axis_proj)
    fpr = fpr / np.max(fpr)
    mask = fpr < thresh
    frac = np.sum(fpr[~mask]) / np.sum(fpr)

    # Copy the 3D mask into the two projected dimensions.
    mask = utils.copy_into_new_dim(mask, (f.shape[axis[0]], f.shape[axis[1]]), axis=-1, copy=True)
    # Put the dimensions in the correct order.        
    isort = np.argsort(list(axis_proj) + list(axis))
    mask = np.moveaxis(mask, isort, np.arange(5))
    
    # Project the masked 5D array onto the specified axis.    
    im = utils.project(np.ma.masked_array(f, mask=mask), axis=axis)
    if return_frac:
        return im, frac
    return im

In [ ]:
def update(thresh, dim1, dim2, log=False, **kws):
    if dim1 == dim2:
        return
    axis = [dims.index(dim1), dims.index(dim2)]
    im, frac = proj2d_mask(f, thresh, normalize=True, return_frac=True, axis=axis)
    
    fig, ax = pplt.subplots()
    kws['norm'] = 'log' if log else None
    kws.setdefault('thresh', 10.0**-3.5)
    kws.setdefault('thresh_type', 'frac')
    kws.setdefault('colorbar', True)
    kws.setdefault('profx', True)
    kws.setdefault('profy', True)
    ax.format(xlabel=dims_units[axis[0]], ylabel=dims_units[axis[1]], title=f'frac = {frac:.2f}')
    mplt.plot_image(im, x=coords[axis[0]], y=coords[axis[1]], ax=ax, prof_kws=prof_kws, **kws)
    ax.format(xlim=sorted(ax.get_xlim()))
    plt.show()
    
    
thresh = widgets.FloatSlider(description='3D thresh', min=0.0, max=0.99, value=0.5, step=0.001)
kws = dict()
interactive(update, thresh=thresh, dim1=dims, dim2=reversed(dims), log=False)

## Radial density

Radial density plots may also convey useful information. We define the radius as $r = \sqrt{\mathbf{x}^T \mathbf{\Sigma}^{-1} \mathbf{x}}$, where $\mathbf{x}$ is the phase space coordinate vector and $\mathbf{\Sigma} = \langle{\mathbf{x}\mathbf{x}^T}\rangle$ is the covariance matrix. So we are looking at the density within nested ellipsoidal shells in the phase space. (This hides information unless the distribution has ellipsoidal symmetry.)

In [ ]:
def get_radii(coords, Sigma):
    COORDS = np.meshgrid(*coords, indexing='ij')
    shape = tuple([len(c) for c in coords])
    R = np.zeros(shape)
    Sigma_inv = np.linalg.inv(Sigma)
    for ii in tqdm(np.ndindex(shape)):
        vec = np.array([C[ii] for C in COORDS])
        R[ii] = np.sqrt(np.linalg.multi_dot([vec.T, Sigma_inv, vec]))
    return R

def radial_density(f, R, radii, dr=None):
    if dr is None:
        dr = 0.5 * np.max(R) / (len(R) - 1)
    fr = []
    for r in tqdm(radii):
        f_masked = np.ma.masked_where(np.logical_or(R < r, R > r + dr), f)
        # I just use the mean density within this shell...
        fr.append(np.mean(f_masked))
    return np.array(fr)

In [ ]:
def plot_radial_density(axis=None, nr=70):
    # Project the distribution onto the specified axis.
    _f = utils.project(f, axis)
    
    # Compute the covariance matrix.
    _coords = [coords[i] for i in axis]
    Sigma, mu = ba.dist_cov(_f, _coords, disp=True)
    
    # Compute the radii in normalized space.
    print('Computing radii on mesh...')
    R = get_radii(_coords, Sigma)
    
    # Compute the approximate radial density in normalized space.
    radii = np.linspace(0.0, np.max(R), nr)
    fr = radial_density(_f, R, radii, dr=None)
    fr = fr / np.nanmax(fr)
    
    # Plot on top of a Gaussian which is normalized in the same way.
    fig, axes = pplt.subplots(ncols=2, figwidth=5.75, figheight=2.55, share=False, space=10)
    for ax in axes:
        alpha = 0.2
        ax.plot(radii, np.exp(-0.5 * radii**2), color='red', alpha=alpha, label='gauss')
        # ax.plot(rs, [float(r <= 2.0) for r in rs], color='blue', alpha=alpha, label='uniform')
        ax.plot(
            radii, fr, color='black', label='data',
            marker='.', lw=0, ms=3,
        )
        ax.legend(ncols=1, loc='upper right', handlelength=1)
    axes.format(xlim=(0.0, 5.0))
    axes[1].format(yscale='log', yformatter='log')
    axes[1].format(ymin=1e-5, ymax=1.5)
    
    title = "-".join([dims[i] for i in axis])
    axes.format(xlabel=r"$r = \sqrt{\mathbf{x}^T\mathbf{\Sigma}^{-1}\mathbf{x}}$", 
                ylabel=r"$f(r)$", title=title)
    return axes

Here are a few examples in 3D. Note the x-y-w distribution, in which the hollowish core is visible.

In [ ]:
ndim = 5  # number of dimensions in projected array
nr = 70  # number of radial points
for axis in itertools.combinations([0, 1, 2, 3, 4], ndim):    
    print('axis =', axis)
    axes = plot_radial_density(axis, nr)
    plt.savefig(f"_output/radial_density_{'-'.join([dims[i] for i in axis])}.png")
    plt.close()

## Corner plots 

### Corner plot with rectangular slices

This shows the full 2D projections.

In [ ]:
def update(cmap=None, log=False):
    axes = mplt.corner(
        f,
        coords=coords,
        diag_kind='None',
        labels=dims_units,
        norm='log' if log else None,
        handle_log='floor',
        thresh=10.0**-3.5,
        thresh_type='frac',
        prof='edges',
        prof_kws=prof_kws,
        linewidth=0, rasterized=True,
        cmap=cmap,
    )
    axes.format(xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
    return axes

interactive(update, cmap=mplt.CMAPS, log=False)

This plot shows everything — all 2D projections — for rectangular slices of the 5D array. (*Very slow*.)

In [ ]:
slice_type = 'int'

# Sliders
n = f.ndim
sliders, checks = [], []
for k in range(n):
    if slice_type == "int":
        slider = widgets.IntSlider(
            min=0,
            max=f.shape[k],
            value=f.shape[k] // 2,
            description=dims[k],
            continuous_update=True,
        )
    elif slice_type == "range":
        slider = widgets.IntRangeSlider(
            value=(0, f.shape[k]),
            min=0,
            max=f.shape[k],
            description=dims[k],
            continuous_update=True,
        )
    else:
        raise ValueError("Invalid `slice_type`.")
    slider.layout.display = "none"
    sliders.append(slider)
    checks.append(widgets.Checkbox(description=f"slice {dims[k]}"))

def hide(button):
    for k in range(n):
        for element in [sliders[k], checks[k]]:
            element.layout.display = None
        if not checks[k].value:
            sliders[k].layout.display = "none"
            
for check in checks:
    check.observe(hide, names="value")
            
mask = np.full(f.shape, False)
_f = np.ma.masked_array(f, mask=mask)
            
def update(
    cmap,
    handle_log,
    log,
    check1,
    check2,
    check3,
    check4,
    check5,
    slider1,
    slider2,
    slider3,
    slider4,
    slider5,
):
    # Make slice that keeps the original dimensions of the array.
    checks = [check1, check2, check3, check4, check5]
    sliders = [slider1, slider2, slider3, slider4, slider5]
    axis_slice = [dims.index(dim) for dim, check in zip(dims, checks) if check]
    ind = sliders
    for k in range(n):
        if type(ind[k]) is int:
            ind[k] = (ind[k], ind[k] + 1)
    idx = utils.make_slice(f.ndim, axis=axis_slice, ind=ind)

    _f.mask[:, :, :, :, :] = True
    _f.mask[idx] = False
    
    # Corner plot of this slice.
    axes = mplt.corner(
        _f,
        coords=coords,
        diag_kind='None',
        labels=dims_units,
        norm='log' if log else None,
        thresh=10.0**-3.5,
        thresh_type='frac',
        prof='edges',
        prof_kws=prof_kws,
        linewidth=0, rasterized=True,
        cmap=cmap,
        handle_log=handle_log,
    )
    axes.format(xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
    return axes

kws = dict()
kws['cmap'] = mplt.CMAPS
kws['handle_log'] = 'floor'
kws['log'] = False
for i, check in enumerate(checks, start=1):
    kws[f"check{i}"] = check
for i, slider in enumerate(sliders, start=1):
    kws[f"slider{i}"] = slider
interactive(update, **kws)

### Corner plot with contour slices 

#### 4D transverse contours: 2D transverse projections and 1D energy projection

Here, we observe the energy distribution within density contours in the 4D transverse phase space.

In [ ]:
ftr = project(f, axis=(0, 1, 2, 3))
ftr = ftr / np.max(ftr)

def update(log=False, thresh=0.5):
    global ftr 
    frac = np.sum(ftr[ftr > thresh]) / np.sum(ftr)
    print(f'frac = {frac:.4f}')
    axes = mplt.corner(
        np.ma.masked_where(ftr < thresh, ftr),
        coords=coords[:4],
        diag_kind='None',
        labels=dims_units,
        thresh=10.0**-3.0,
        thresh_type='frac',
        fill_value=0.0,
        norm='log' if log else None,
        handle_log='floor',
        prof='edges',
        prof_kws=dict(kind='step', lw=0.4, alpha=0.8),
        linewidth=0,rasterized=True
    )
    pw = proj1d_mask(f, thresh, fpr=ftr, normalize=True, return_frac=False, axis=4)
    ax = axes[0, 2]
    for i in range(3):
        for j in range(i + 1):
            axes[i, j]._shared_x_axes.remove(ax)
            axes[i, j]._shared_y_axes.remove(ax)
    ax.axis('on')
    ax.bar(coords[4], pw, color='black')
    ax.format(
        xlim=(np.min(coords[4]), np.max(coords[4])),
        ylim=(0, 0.1),
        xspineloc='bottom', yspineloc='left',
        title='energy projection',
    )
    return axes, frac
    
interactive(update, log=False, thresh=(0.0, 0.99, 0.01))

Plot 2D projections of 4D phase space in energy slices.

In [ ]:
def update(log, w):
    axes = mplt.corner(
        f[..., w],
        coords=coords[:4],
        diag_kind='None',
        labels=dims_units,
        thresh=10.0**-3.5,
        thresh_type='frac',
        fill_value=0.0,
        norm='log' if log else None,
        handle_log='floor',
        prof='edges',
        prof_kws=dict(kind='step', lw=0.4, alpha=0.8),
        linewidth=0,rasterized=True
    )
    axes.format(xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
    return axes
    
interactive(update, log=False, w=widgets.IntSlider(description='w', min=0, max=len(coords[4]) - 1))

#### 4D transverse contours: all projections

Plot the 2D projections of the 5D phase space distribution as the 4D boundary is changed in transverse phase space. To slice the array, we compute a 4D mask in the transverse phase space, then copy the mask along the last axis of the array to get a 5D mask. *(This is slow.)*. The only thing new, relative to the previous plot, is the 2D correlations of energy with the other variables.

In [ ]:
def mask_4d(f, thresh=0.0, ftr=None):
    """Mask N-D array `f` based on contours of `f.sum(axis=-1)`."""
    if ftr is None:
        ftr = np.sum(f, axis=-1)
        ftr = ftr / np.max(ftr)
    condition = utils.copy_into_new_dim(ftr < thresh, f.shape[-1])
    return np.ma.masked_where(condition, f)

In [ ]:
def update(log, thresh):
    axes = mplt.corner(
        mask_4d(f, thresh, ftr=ftr),
        coords=coords,
        diag_kind='None',
        labels=dims_units,
        thresh=10.0**-3.0,
        thresh_type='frac',
        fill_value=0.0,
        norm='log' if log else None,
        handle_log='floor',
        prof='edges',
        prof_kws=dict(kind='step', lw=0.4, alpha=0.8),
    )
    plt.show()
    
thresh = widgets.FloatSlider(description="x-x'-y-y' thresh", min=0.0, max=0.99, value=0.5, step=0.001)
interactive(update, log=False, thresh=thresh)

#### 5D contours

Plot the 2D projections of the 5D array within 5D density contours.

In [ ]:
def update(cmap, log, thresh):
    axes = mplt.corner(
        np.ma.masked_less_equal(f, thresh),
        coords=coords,
        diag_kind='None',
        labels=dims_units,
        norm='log' if log else None,
        handle_log='floor',
        thresh=10.0**-3.5,
        thresh_type='frac',
        prof='edges',
        prof_kws=prof_kws,
        linewidth=0, rasterized=True,
        cmap=cmap,
    )
    axes.format(xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
    return axes

thresh = widgets.FloatSlider(description="5D thresh", min=0.0, max=0.99, value=0.5, step=0.001)
interactive(update, cmap=mplt.CMAPS, log=False, thresh=thresh)

View the x-w projection within shrinking 5D density contours.

In [ ]:
threshs = np.linspace(0.0, 0.7, 12)
ims = []
for thresh in tqdm(threshs):
    im = utils.project(np.ma.masked_less_equal(f, thresh), axis=(4, 0))
    ims.append(im)

In [ ]:
ncols = 6
nrows = int(np.ceil(len(ims) / ncols))
fig, axes = pplt.subplots(nrows=nrows, ncols=ncols, figwidth=8)
axes.format(xlabel=dims_units[4], ylabel=dims_units[0])
for ax, im, thresh in zip(axes, ims, threshs):
    mplt.plot_image(im, x=coords[4], y=coords[0], ax=ax)
    ax.annotate(f"t = {thresh:.2f}", xy=(0.03, 0.03), xycoords='axes fraction', 
                color='red', fontsize='small')
plt.show()